In [17]:
import pandas as pd
import json
from statistics import mean

In [18]:
df = pd.read_json("./diary.json")

In [19]:
df.head()

,name,quantity,date
0,Frutta fresca,1,2023-09-19
1,Pasta al pomodoro,1,2023-09-19
2,Nasello gratinato,1,2023-09-19
3,Insalata verde,4,2023-09-19
4,Frutta fresca,1,2023-09-20


In [21]:
with open("./diary.json", "r") as file:
    diary = json.load(file)

with open("../src/static/tags.json", "r") as file:
    all_tags = json.load(file)

In [37]:
COMMON_FOODS_CONSTANT = 3
MINUMUM_VOTES = 1

def quantity_to_vote(quantity):
    return [10, 7, 3, 0][quantity - 1]

def compute_vote(all_qty, total_avg):
    if not all_qty:
        return -1
    avg = mean(all_qty)
    votes_part = len(all_qty) / (len(all_qty) + COMMON_FOODS_CONSTANT) * avg
    population_part = COMMON_FOODS_CONSTANT / (len(all_qty) + COMMON_FOODS_CONSTANT) * total_avg
    return votes_part + population_part

In [38]:
meals = []
total_avg = mean([quantity_to_vote(de["quantity"]) for de in diary])
for meal, tags in all_tags.items():
    diary_entries = [entry for entry in diary if entry["name"].lower() == meal]
    all_qty = [quantity_to_vote(de["quantity"]) for de in diary_entries]
    avg = mean(all_qty) if all_qty else -1
    vote = compute_vote(all_qty, total_avg)
    meals.append({ "tags": tags, "avg": avg, "vote": vote, "meal": meal })

meals

[{'tags': ['pasta', 'primo', 'carboidrati'],
  'avg': 0,
  'vote': -1,
  'meal': "pasta con gli aromi d'estate"},
 {'tags': ['uova', 'parmigiano', 'secondo', 'proteine'],
  'avg': 10,
  'vote': 9.229972274916095,
  'meal': 'frittata'},
 {'tags': ['fagiolini', 'contorno', 'verdura'],
  'avg': 0,
  'vote': -1,
  'meal': "fagiolini all'olio"},
 {'tags': ['frutta'],
  'avg': 10,
  'vote': 8.588282504012842,
  'meal': 'frutta fresca'},
 {'tags': ['pane', 'carboidrati'], 'avg': 0, 'vote': -1, 'meal': 'pane'},
 {'tags': ['riso', 'parmigiano', 'primo', 'carboidrati'],
  'avg': 3,
  'vote': 6.132423756019262,
  'meal': 'risotto alla parmigiana'},
 {'tags': ['melanzane', 'verdura', 'contorno'],
  'avg': 0,
  'vote': 5.382423756019262,
  'meal': 'melanzane'},
 {'tags': ['zucchine', 'secondo', 'verdura'],
  'avg': 0,
  'vote': 5.382423756019262,
  'meal': 'sformato di zucchine'},
 {'tags': ['pasta', 'pomodoro', 'primo', 'verdura', 'carboidrati'],
  'avg': 7.888888888888889,
  'vote': 7.71080791867

In [ ]:
all_used_tags = list({ tag for meal, tags in all_tags.items() for tag in tags })

In [ ]:
meals

In [39]:
pd.DataFrame(meals)

,tags,avg,vote,meal
0,"[pasta, primo, carboidrati]",0.00,-1.000000,pasta con gli aromi d'estate
1,"[uova, parmigiano, secondo, proteine]",10.00,9.229972,frittata
2,"[fagiolini, contorno, verdura]",0.00,-1.000000,fagiolini all'olio
3,[frutta],10.00,8.588283,frutta fresca
4,"[pane, carboidrati]",0.00,-1.000000,pane
...,...,...,...,...
126,"[zucca, contorno, verdura]",0.75,3.504242,zucca al forno
127,"[verza, mela, contorno, verdura]",0.00,2.691212,verza stufata con mele renette a pezzetti e ch...
128,"[verza, mela, contorno, verdura]",0.00,-1.000000,verza stufata con mele renette a pezzi e chiod...
129,"[spinacino, verdura, contorno]",0.00,2.392188,insalata spinacino
